In [9]:
%cd /workspace

from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm

/workspace


In [10]:
INPUT = Path("/workspace/resources/input")
task_df = pd.read_parquet(INPUT / "task2_dataset_raw_train.parquet")
poi_df = pd.read_parquet(INPUT / "cell_POIcat.parquet")

# task_df = pd.concat([task_df] * 4)

### GCN Examples

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Graph Convolution 層の定義
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        return output


# Mesh GCN モデルの定義
class MeshGCN(nn.Module):
    def __init__(self, feature_dim, hidden_dim, num_classes):
        super(MeshGCN, self).__init__()
        self.gc1 = GraphConvolution(feature_dim, hidden_dim)
        self.gc2 = GraphConvolution(hidden_dim, num_classes)

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = self.gc2(x, adj)
        return x


# メッシュの特徴データと隣接行列の準備
mesh_features = torch.rand((4, 10))
adjacency_matrix = torch.tensor(
    [
        [0, 1, 0, 0],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
    ],
    dtype=torch.float32,
)

# ユーザーのメッシュ位置のインデックス
user_indices = torch.tensor([0, 2, 3])  # user1はm1, user2はm3, user3はm4に位置

# ユーザーデータの取得
user_data = mesh_features[user_indices]  # features

# ユーザーに対応する隣接行列の部分の取得
user_adj = adjacency_matrix[user_indices][:, user_indices]

# モデルのインスタンス化と予測の実行
model = MeshGCN(10, 64, 1)
updated_features = model(user_data, user_adj)

print(updated_features)

tensor([[0.0000],
        [0.2945],
        [0.5518]], grad_fn=<MmBackward0>)


In [16]:
mesh_features.shape

torch.Size([4, 10])

In [17]:
user_indices

tensor([0, 2, 3])

In [18]:
user_data.shape

torch.Size([3, 10])

In [19]:
user_adj

tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.]])

In [20]:
updated_features.shape

torch.Size([3, 1])

### Transformer Example

In [74]:
import torch
import torch.nn as nn


class CustomTransformerModelV1(nn.Module):
    def __init__(
        self,
        input_size1,
        input_size2,
        d_model,
        output_size,
        nhead=8,
        num_encoder_layers=6,
        num_decoder_layers=6,
    ):
        super().__init__()
        self.embedding_src = nn.Linear(input_size1, d_model)
        self.embedding_tgt = nn.Linear(input_size2, d_model)
        
        self.transformer = nn.Transformer(
            d_model,
            nhead,
            num_encoder_layers,
            num_decoder_layers,
            batch_first=True,
        )
        self.out = nn.Linear(d_model, output_size)

    def forward(self, batch):
        x_src = self.embedding_src(batch["feature_seqs"])
        x_tgt = self.embedding_tgt(batch["auxiliary_seqs"])

        src_mask = batch["feature_padding_mask"]
        tgt_mask = batch["auxiliary_padding_mask"]

        x = self.transformer(
            src=x_src,
            tgt=x_tgt,
            src_key_padding_mask=src_mask,
            tgt_key_padding_mask=tgt_mask,
        )
        x = self.out(x)

        return x


# ハイパーパラメータ
batch_size = 32
input_size1 = 5
input_size2 = 12
d_model = 128
seq_len_src = 20
seq_len_tgt = 10
output_size = 1
nhead = 2

# モデルのインスタンス化
model = CustomTransformerModelV1(nhead=nhead, input_size1=input_size1, input_size2=input_size2, d_model=d_model, output_size=output_size)

# 入力データの準備
batch = {
    "feature_seqs": torch.randn(batch_size, seq_len_src, input_size1),
    "auxiliary_seqs": torch.randn(batch_size, seq_len_tgt, input_size2),
    "feature_padding_mask": (torch.randn(batch_size, seq_len_src) > 0.5),  # 仮のマスク
    "auxiliary_padding_mask": (torch.randn(batch_size, seq_len_tgt) > 0.5),  # 仮のマスク
}
# モデルの実行
output = model(batch)
print(output.shape)  # [batch_size, seq_len_tgt, output_size] の形になるはず

torch.Size([32, 10, 1])
